In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data ='/content/6671501a18c55_round2_input.xlsx'

In [ ]:
batting_df = pd.read_excel(data, sheet_name='batsman_scorecard')
bowling_df = pd.read_excel(data, sheet_name='bowler_scorecard')
matches_df = pd.read_excel(data, sheet_name='matchlevel_scorecard')

In [ ]:
player_ids = ['325814','1626526','1506098','82228','8250115','4171460','2275195','5652758','8986193','8246573','7758638']

# Convert player_ids to integers if necessary
player_ids = [int(id) for id in player_ids]

# Create the boolean mask
data_subset = (batting_df['batsman_id'].isin(player_ids)) | (bowling_df['bowler_id'].isin(player_ids))
#data_batsman = data['batsman_id'].isin(player_ids)

In [ ]:
filtered_df = batting_df[batting_df['batsman_id'].isin([int(id) for id in player_ids])]


total_runs = filtered_df.groupby('batsman_id')['runs'].sum()
total_runs = total_runs[total_runs > 100]

# Step 3: Calculate centuries and half-centuries
centuries_half_centuries = filtered_df.assign(
    centuries=lambda x: (x['runs'] >= 100).astype(int),
    half_centuries=lambda x: ((x['runs'] >= 50) & (x['runs'] < 100)).astype(int)
).groupby('batsman_id').agg({
    'centuries': 'sum',
    'half_centuries': 'sum'
})

#  Extract strike rate
strike_rates = filtered_df.groupby('batsman_id')['strike_rate'].max()

#  Calculate batting average (total runs / number of innings)
# Assuming each row in the filtered_df represents one inning
innings_counts = filtered_df.groupby('batsman_id').size()
batting_average = total_runs / innings_counts

#  Combine all statistics into a new DataFrame
combined_batsman_df = pd.DataFrame({
    'total_runs': total_runs,
    'centuries': centuries_half_centuries['centuries'],
    'half_centuries': centuries_half_centuries['half_centuries'],
    'strike_rate': strike_rates,
    'batting_avg': batting_average
}).sort_values(by='total_runs', ascending=False)

print(combined_batsman_df)

            total_runs  centuries  half_centuries  strike_rate  batting_avg
batsman_id                                                                 
1626526         2737.0          1              17       300.00    30.411111
2275195         2480.0          3              14       400.00    26.105263
82228            874.0          1               2       280.00    27.312500
325814           822.0          1               7       225.81    30.444444
8250115          700.0          0               6       161.54    33.333333
1506098          677.0          1               3       216.67    30.772727
4171460          400.0          0               0       200.00    14.814815
5652758            NaN          0               0       166.67          NaN
7758638            NaN          0               0       100.00          NaN
8246573            NaN          0               0         0.00          NaN
8986193            NaN          0               0        75.00          NaN


In [ ]:
def calculate_points(row):
    # Points for Strike Rate
    if row['strike_rate'] >= 150:
        strike_rate_points = 50
    elif row['strike_rate'] >= 100:
        strike_rate_points = 40
    elif row['strike_rate'] >= 80:
        strike_rate_points = 30
    else:
        strike_rate_points = 0

    # Points for Average
    if row['batting_avg'] >= 50:
        avg_points = 30
    elif row['batting_avg'] >= 40:
        avg_points = 20
    elif row['batting_avg'] >= 30:
        avg_points = 10
    else:
        avg_points = 5

    # # Points for 100s
    if row['centuries'] >= 3:
        century_points = 30
    elif row['centuries'] == 2:
        century_points = 20
    elif row['centuries'] == 1:
        century_points = 10
    else:
        century_points = 0

    # Points for 50s
    if row['half_centuries'] >= 5:
        half_century_points = 20
    elif row['half_centuries'] >= 3:
        half_century_points = 10
    elif row['half_centuries'] >= 1:
        half_century_points = 5
    else:
        half_century_points = 0

    # Total points
    return strike_rate_points + avg_points + century_points + half_century_points

combined_batsman_df['batsmen_points'] = combined_batsman_df.apply(calculate_points, axis=1)

combined_batsman_df.sort_values(by = 'batsmen_points',ascending = False)

,total_runs,centuries,half_centuries,strike_rate,batting_avg,batsmen_points
batsman_id,,,,,,
2275195,2480.0,3,14,400.00,26.105263,105
1626526,2737.0,1,17,300.00,30.411111,90
325814,822.0,1,7,225.81,30.444444,90
8250115,700.0,0,6,161.54,33.333333,80
1506098,677.0,1,3,216.67,30.772727,80
82228,874.0,1,2,280.00,27.312500,70
4171460,400.0,0,0,200.00,14.814815,55
5652758,NaN,0,0,166.67,NaN,55
7758638,NaN,0,0,100.00,NaN,45


In [ ]:
[1626526,325814]

In [ ]:
player_ids = [1626526, 325814]

# Step 1: Filter the DataFrame for relevant player_ids
filtered_df = batting_df[batting_df['batsman_id'].isin(player_ids)]

# Step 2: Calculate the strike rate for each match (runs scored per 100 balls faced)
filtered_df['strike_rate'] = (filtered_df['runs'] / filtered_df['balls_faced']) * 100

# Step 3: Determine centuries and half-centuries
filtered_df['centuries'] = (filtered_df['runs'] >= 100).astype(int)
filtered_df['half_centuries'] = ((filtered_df['runs'] >= 50) & (filtered_df['runs'] < 100)).astype(int)
filtered_df['batting_avg'] = filtered_df['runs'] / filtered_df['balls_faced']
# Step 4: Calculate the batting average for each match (runs scored per match)
# Note: Batting average is typically calculated over a series of innings, but here we display runs scored per match.

# Step 5: Combine these statistics into a new DataFrame
combined_df = filtered_df[['batsman_id', 'match id', 'match_dt', 'runs', 'balls_faced', 'strike_rate','batting_avg', 'centuries', 'half_centuries']]

combined_df['batsmen_points'] = combined_df.apply(calculate_points, axis=1)

combined_df.sort_values(by = 'batsmen_points',ascending = False)

<ipython-input-41-ccba990d54e3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['strike_rate'] = (filtered_df['runs'] / filtered_df['balls_faced']) * 100
<ipython-input-41-ccba990d54e3>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['centuries'] = (filtered_df['runs'] >= 100).astype(int)
<ipython-input-41-ccba990d54e3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

,batsman_id,match id,match_dt,runs,balls_faced,strike_rate,batting_avg,centuries,half_centuries,batsmen_points
7090,1626526,8887535,2021-12-11,114,73,156.164384,1.561644,1,0,65
13097,325814,9085439,2022-07-03,100,47,212.765957,2.127660,1,0,65
12900,325814,9085327,2022-06-24,70,31,225.806452,2.258065,0,1,60
2465,1626526,8729209,2021-06-11,90,36,250.000000,2.500000,0,1,60
13871,1626526,9094343,2022-08-20,67,37,181.081081,1.810811,0,1,60
...,...,...,...,...,...,...,...,...,...,...
2656,1626526,8729300,2021-06-21,0,1,0.000000,0.000000,0,0,5
16451,1626526,9262175,2022-09-18,1,4,25.000000,0.250000,0,0,5
20217,1626526,9433577,2023-06-18,0,3,0.000000,0.000000,0,0,5
20156,325814,9433542,2023-06-18,0,1,0.000000,0.000000,0,0,5


In [ ]:
combined_df = combined_df.sort_values(by=['batsman_id', 'match_dt'],ascending = [True,False])

# Define exponentially decaying weights
def generate_exponential_weights(n, decay_factor=0.9):
    return [decay_factor ** (n - i - 1) for i in (range(n))]

# Function to calculate weighted moving average
def weighted_moving_average(points, weights):
    return sum(p * w for p, w in zip(points, weights)) / sum(weights)

# Calculate WMA for each player
def calculate_wma_for_player(player_df, window_size, decay_factor=0.9):
    weights = generate_exponential_weights(window_size, decay_factor)
    player_df['wma'] = player_df['batsmen_points'].rolling(window=window_size).apply(
        lambda x: weighted_moving_average(x, weights) if len(x) == window_size else np.nan, raw=False)
    return player_df

# Define the window size
window_size = 5

# Apply the function to each player
combined_df = combined_df.groupby('batsman_id', group_keys=False).apply(calculate_wma_for_player, window_size)
combined_df.sort_values(by='wma',ascending = False).head(10)

,batsman_id,match id,match_dt,runs,balls_faced,strike_rate,batting_avg,centuries,half_centuries,batsmen_points,wma
12599,325814,9085152,2022-06-19,55,34,161.764706,1.617647,0,1,60,57.834119
17065,1626526,9272507,2022-12-29,98,53,184.905660,1.849057,0,1,60,57.111060
21493,1626526,9484502,2023-02-16,58,28,207.142857,2.071429,0,1,60,57.111060
17167,1626526,9272570,2023-01-04,38,14,271.428571,2.714286,0,0,55,56.899954
21562,1626526,9484558,2023-02-23,9,5,180.000000,1.800000,0,0,55,55.988987
17152,1626526,9272549,2023-01-01,10,6,166.666667,1.666667,0,0,55,55.988987
21632,1626526,9484607,2023-03-03,11,5,220.000000,2.200000,0,0,55,54.496716
13065,1626526,9085418,2022-07-03,27,14,192.857143,1.928571,0,0,55,53.823106
13718,1626526,9094231,2022-08-06,36,22,163.636364,1.636364,0,0,55,53.692340
12711,325814,9085215,2022-06-21,73,37,197.297297,1.972973,0,1,60,53.588069


## **BOWLERS**

In [ ]:
filtered_df = bowling_df[bowling_df['bowler_id'].isin([int(id) for id in player_ids])]

#  Calculate total wickets, balls bowled, runs conceded, and 4-wicket hauls
total_wickets = filtered_df.groupby('bowler_id')['wicket_count'].sum()
total_balls_bowled = filtered_df.groupby('bowler_id')['balls_bowled'].sum()
total_runs_conceded = filtered_df.groupby('bowler_id')['runs'].sum()
four_wicket_hauls = filtered_df.assign(
    four_wickets=lambda x: (x['wicket_count'] >= 4).astype(int)
).groupby('bowler_id')['four_wickets'].sum()

#  Calculate the strike rate (balls bowled per wicket)
strike_rate = total_balls_bowled / total_wickets

#  Calculate the economy rate (runs conceded per over)
economy_rate = total_runs_conceded / (total_balls_bowled / 6)

#  Calculate the bowling average (runs conceded per wicket)
bowling_average = total_runs_conceded / total_wickets

#  Combine all statistics into a new DataFrame
combined_bowler_df = pd.DataFrame({
    'total_wickets': total_wickets,
    'balls_bowled': total_balls_bowled,
    'runs_conceded': total_runs_conceded,
    'strike_rate': strike_rate,
    'economy_rate': economy_rate,
    'Average': bowling_average,
    '4w': four_wicket_hauls
}).sort_values(by='total_wickets', ascending=False)

print(combined_bowler_df)

           total_wickets  balls_bowled  runs_conceded  strike_rate  \
bowler_id                                                            
5652758               44           623            860    14.159091   
7758638               40           951           1137    23.775000   
2275195               36           882           1154    24.500000   
8986193               22           283            338    12.863636   
4171460               21           462            631    22.000000   
8246573               14           151            167    10.785714   
82228                  5           132            162    26.400000   
325814                 2            60             90    30.000000   
1626526                0             6             10          inf   

           economy_rate    Average  4w  
bowler_id                               
5652758        8.282504  19.545455   2  
7758638        7.173502  28.425000   0  
2275195        7.850340  32.055556   0  
8986193        7.166078 

In [ ]:
def calculate_points_bowler(row):
    points = 0

    # Strike Rate points
    if row['strike_rate'] <= 15:
        points += 30
    elif 15 < row['strike_rate'] <= 19:
        points += 20
    elif 19 < row['strike_rate'] <= 24:
        points += 10
    # No points for Strike Rate > 24

    # Economy points
    if row['economy_rate'] <= 3:
        points += 50
    elif 3 < row['economy_rate'] <= 5:
        points += 40
    elif 5 < row['economy_rate'] <= 7:
        points += 30
    # No points for Economy >= 7

    # Average points
    if row['Average'] <= 20:
        points += 30
    elif 20 < row['Average'] <= 30:
        points += 20
    elif 30 < row['Average'] <= 40:
        points += 10
    # No points for Avg > 40

    # 4w per innings points
    if row['4w'] >= 4:
        points += 30
    elif row['4w'] == 2 or row['4w'] == 3:
        points += 20
    elif row['4w'] == 1:
        points += 10
    # No points for 4w per innings == 0

    return points

# Apply the function to calculate points for each row
combined_bowler_df['bowler_points'] = combined_bowler_df.apply(calculate_points_bowler, axis=1)
combined_bowler_df.sort_values(by='bowler_points',ascending = False).head(15)

,total_wickets,balls_bowled,runs_conceded,strike_rate,economy_rate,Average,4w,bowler_points
bowler_id,,,,,,,,
8246573,14,151,167,10.785714,6.635762,11.928571,1,100
5652758,44,623,860,14.159091,8.282504,19.545455,2,80
8986193,22,283,338,12.863636,7.166078,15.363636,2,80
4171460,21,462,631,22.000000,8.194805,30.047619,2,40
7758638,40,951,1137,23.775000,7.173502,28.425000,0,30
2275195,36,882,1154,24.500000,7.850340,32.055556,0,10
82228,5,132,162,26.400000,7.363636,32.400000,0,10
325814,2,60,90,30.000000,9.000000,45.000000,0,0
1626526,0,6,10,inf,10.000000,inf,0,0
